Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [63]:
import logging
import sys
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import math
import numpy as np
from tqdm import tqdm

## The *Nim* and *Nimply* classes

In [64]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [65]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [66]:
def pure_random(state: Nim, genomes = None) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [67]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [68]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [69]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows]) #trasforma il numero in binario
    xor = np.bitwise_xor.reduce(tmp, axis=0)
    n = int("".join(str(_) for _ in xor), base=2) #ritrasforma il numero da binario in decimale
    return n


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    #print(cooked)
    return cooked


def optimal(state: Nim, genomes=None) -> Nimply:
    analysis = analize(state)
    #logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Strategies

In [70]:
def find_genomes():
    pre = np.random.normal(loc=0, scale=1, size=2) #Create two different gaussian variables
    val = 0.5 * (1 + np.tanh(pre / np.sqrt(2)))
    print(val)
    return val
    
def choose_element(vector, alpha):
    alpha = max(0, min(alpha, 1)) #Ensure that alpha is between 0 and 1
    index = int(alpha * (len(vector)))# Calculate the index based on the value of alpha
    if index >= len(vector)-1:
        index = len(vector) - 1
    if index < 0:
        index = 0
    chosen_element = vector[index]# Return the corresponding element
    return chosen_element

find_genomes()

[0.16860179 0.92181369]


array([0.16860179, 0.92181369])

In [71]:
nim = Nim(3)
genomes = find_genomes()

def adaptive(state: Nim, genomes) -> Nimply:
    """Is it a random move?"""
    indices = [i for i, elemento in enumerate(state.rows) if elemento != 0]
    row = choose_element(indices, genomes[0])
    closest_genome = genomes[0]
    while state.rows[row] == 0:
        closest_genome += random.uniform(-0.25, 0.25)
        row = choose_element(indices, closest_genome)
    if (state.rows[row] == 1):
        num_objects = 1
    else:
        num_objects = choose_element(range(1, state.rows[row]), genomes[1])
    return Nimply(row, num_objects)

[0.69113873 0.38105492]


In [72]:
def play_game(strat, genomes):
    #logging.getLogger().setLevel(logging.INFO)

    strategy = (strat, optimal)

    nim = Nim(3)
    #logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim, genomes)
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
    #logging.info(f"status: Player {player} won!")
    return player

#play_game(adaptive, genomes)

# Fitness function
def fitness(strategy, genomes, num_games=100):
    wins = 0
    for _ in range(num_games):
        if play_game(strategy, genomes):
            wins += 1
    return (num_games - wins) / num_games

In [73]:
def tweak(genome, mu=0, sigma=np.array([0.1, 0.1])):
    dist = np.array([np.random.normal(mu, sigma[0], 1), np.random.normal(mu, sigma[1], 1)])
    new = np.array([0,0])

    new[0] = genome[0] + dist[0][0]
    new[1] = genome[1] + dist[1][0]

    if new[0] >= 1:
        new[0] = 0.999
    elif new[0] < 0:
        new[0] = 0

    if new[1] >= 1:
        new[1] = 0.999
    elif new[1] < 0:
        new[1] = 0
    
    return new

In [74]:
def train(generations=100, population_size=100, mu=0, sigma=0.1, save_parent=True):
    genomes = find_genomes()
    parent_genome = (genomes, fitness(adaptive, genomes))

    for _ in tqdm(range(generations), file=sys.stdout):
        best_genome = ({"row": 0, "elements": 0}, 0)
        for i in range(population_size):
            
            child_genome = tweak(parent_genome[0], mu, np.array([sigma, sigma]))
            fit = fitness(adaptive, child_genome)
            if fit > best_genome[1] or i == 0:
                best_genome = (child_genome, fit)

        if save_parent: #it will depend if it's a (1,lambda) or a (1+lambda) situation
            if best_genome[1] > parent_genome[1]:
                parent_genome = best_genome
                print(parent_genome)
        else:
            parent_genome = best_genome
            print(parent_genome)

In [75]:
train(generations=100, population_size=100, mu=0.1, sigma=0.2, save_parent=True)

[0.73252718 0.2866656 ]
100%|██████████| 100/100 [08:56<00:00,  5.37s/it]
